In [5]:
import math
import random
import torch
import matplotlib.pyplot as plt
from scipy.spatial import distance
import itertools
import jellyfish
import numpy as np

In [11]:
# import the words I'm using
%run partial_words.py
%run core.py
%run imgCls.py

In [12]:
rt_name = "mdl_spelling"
dcMapping = {}
maxDepth = 30
wrl = WordManage()
wrl.set_words(sbst,0)
train_and_choose(rt_name,wrl,dcMapping,maxDepth)
save_json(dcMapping,"word_tree_mapping.json")

train tree..
category  0  total word count  54
recursing on  mdl_spelling_0  total words..  54
train tree..
category  0  total word count  19
recursing on  mdl_spelling_0_0  total words..  19
train tree..
category  0  total word count  2
recursing on  mdl_spelling_0_0_0  total words..  2
Words.. ['understanding', 'advertising']
train tree..
category  0  total word count  1
category  1  total word count  1
category  2  total word count  0
category  3  total word count  0
category  4  total word count  0
category  1  total word count  2
recursing on  mdl_spelling_0_0_1  total words..  2
Words.. ['entertainment', 'engineering']
train tree..
category  0  total word count  1
category  1  total word count  1
category  2  total word count  0
category  3  total word count  0
category  4  total word count  0
category  2  total word count  2
recursing on  mdl_spelling_0_0_2  total words..  2
Words.. ['philosophy', 'opportunity']
train tree..
category  0  total word count  1
category  1  total wo

In [ ]:
def test_words(wrds):
    wrl = WordManage()
    wrl.set_words(wrds,0)
    mdl = build_choose_and_train(wrl,dbg=True,out_cat=5)
    wrds = wrl.all_words()
    cats = get_category(mdl,wrds,dbg=True)
    print("categories... ",cats)
    return

In [ ]:
im = ImgNet(5)
im.apply(custom_init_weights)
im = best_move(im)
get_category(im,['Chris','Schrumm','Programmer','Test','Nice','Some','Zero','Whatever','Mouse','Other'])

In [ ]:
ww = word_batch(['Chris','Schrumm'])

In [ ]:
def display_im(im):
    plt.figure(figsize=(25, 150),dpi=80)
    plt.imshow(im)

In [ ]:
display_im(ww[0][0])

In [ ]:
tt = numpy_to_tensor(np.array([3,4,5]))
tt.to(best_device())
tensor_info(tt)

In [ ]:
word_batch(["chris","schrumm"]).shape

In [ ]:
wrl = WordManage()
wrl.set_words(sbst,0)
ntwrk_name = "mdl_spelling"
dr = "./net_data"
fl = dr + "/" + ntwrk_name + ".bin"
mdl = best_move(ImgNet(5))
load_file_into_model(mdl,fl)
all_wrds = wrl.all_words()
cat_w = get_category(mdl,all_wrds)
out_cat = 5

In [ ]:
build_choose_and_train(wrl,dbg=True)

In [ ]:
cat_list = [[all_wrds[v] for v,ct in enumerate(cat_w) if ct == i ] for i in range(out_cat)]

In [ ]:
targ_arr = []
in_arr = []

for _ in range(15):
    for i in range(out_cat):
        #print("lenght list ",len(cat_list[i]))
        lst = random.sample(cat_list[i], k=min(len(cat_list[i]),15))
        #print(len(lst))
    
        for w in lst:
            targ_arr.append(i)
            wd = word_mix(w)
            wrd = encode_word(wd)
            wrd = np.expand_dims(wrd,axis=0)
            in_arr.append(wrd)
            
tn_in, tn_trg  = np.stack(in_arr),np.array(targ_arr,dtype=np.long)


In [ ]:
l = tn_trg.shape[0]

In [ ]:
l

In [ ]:
def train_loop(epoch,tn_in,tn_arg, model, optimizer, scheduler,dbg):   
    
    example_size = tn_arg.shape[0]
    example_indexes = [x for x in range(example_size)]
    for i in range(epoch):
        for b in batch(example_indexes,256):
            #print(b[0],b[-1])
            
            optimizer.zero_grad()
            
            data = tn_in[b[0]:b[-1]]
            target = tn_arg[b[0]:b[-1]]
            
            #print("training data ",data.shape)
            
            data = numpy_to_tensor(data)
            target = numpy_to_tensor(target)
            
            output = model(data)
            
            loss = F.nll_loss(output,target)
            loss.backward()
            
            optimizer.step()
            
        scheduler.step()
        if dbg:
            print("lr.. ",scheduler.get_lr())

In [ ]:
'''
def build_choose_and_train(wrl,dbg=False,out_cat=5):
    out_num = 5
    targ_arr = []
    in_arr = []
    wrds = choose_spread_combo(out_num, wrl)
    
    for en,v in enumerate(wrds):
        targ_arr.append(en)
        wrd = encode_word(v)
        wrd = np.expand_dims(wrd,axis=0)
        in_arr.append(wrd)
        if dbg:
            print("word ",v," category ",en)
      
    
    for i in range(1000):
        for en,v in enumerate(wrds):
            targ_arr.append(en)
            wd = word_mix(v)
            #if dbg:
            #    print("word ",v," mix ",wd," category ",en)
            wrd = encode_word(wd)
            wrd = np.expand_dims(wrd,axis=0)
            in_arr.append(wrd)
                        
            
    if dbg:
        print("length of input.. ",len(in_arr))
        
    tn_in, tn_trg  = np.stack(in_arr),np.array(targ_arr,dtype=np.long)
    
    epoch = 6
    
    # example_size = len(targ_arr)
    # example_indexes = [x for x in range(example_size)]
    
    model, optimizer, scheduler = build_net_opt_schedule(out_cat)
    
    train_loop(epoch,tn_in,tn_trg, model, optimizer, scheduler,dbg)
    
    
    if dbg:
        print("second half training..")
    
    # choose category based on the partially trained model..
    
    all_wrds = wrl.all_words()
    cat_w = get_category(model,all_wrds)
    
    #for i,ct in enumerate(cat_w):
    cat_list = [[all_wrds[v] for v,ct in enumerate(cat_w) if ct == i ] for i in range(out_cat)]
    
    in_arr = []
    targ_arr = []
    
    epoch = 40
    
    for _ in range(30):
        for i in range(out_cat):
            #print("lenght list ",len(cat_list[i]))
            lst = random.sample(cat_list[i], k=min(len(cat_list[i]),15))
            #print(len(lst))
    
            for w in lst:
                targ_arr.append(i)
                wd = word_mix(w)
                wrd = encode_word(wd)
                wrd = np.expand_dims(wrd,axis=0)
                in_arr.append(wrd)
            
    tn_in, tn_trg  = np.stack(in_arr),np.array(targ_arr,dtype=np.long)
    
    train_loop(epoch,tn_in,tn_trg, model, optimizer, scheduler,dbg)
    
    if dbg:
        model.eval()
        with torch.no_grad():
            example_size = tn_trg.shape[0]
            example_indexes = [x for x in range(example_size)]
            
            for b in batch(example_indexes,256):
                data = tn_in[b[0]:b[-1]]
                target = tn_trg[b[0]:b[-1]]
                
                data = numpy_to_tensor(data)
                target = numpy_to_tensor(target)
                output = model(data)
                
                # test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
                pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
                correct = pred.eq(target.view_as(pred)).sum().item()
                
                print("correct on batch.. ", (100.0 * correct) / len(b))
    return model
'''

In [128]:
data_ky = load_json("word_tree_mapping.json")

In [ ]:
ntwrk_name = "mdl_spelling"
wrd = "marketing"
dr = "./net_data"

mdl = best_move(ImgNet(5))
       
fl = dr + "/" + ntwrk_name + ".bin"
    
nt = torch.load(fl)

mdl.load_state_dict(nt)
mdl.eval()
      
cats = get_category(mdl,[wrd])

In [ ]:
cats

In [ ]:
mmp = data_ky
sb_ntwrk = ntwrk_name + "_" +  str(cats[0])

word = [k for k,v in mmp.items() if v == sb_ntwrk]

In [ ]:
sb_ntwrk

In [140]:
# pell_word(wrd,ntwrk_name,mmp,dr, dbg=False):     
spell_word("dibate","mdl_spelling",data_ky,"./net_data",dbg=True)

category..  [0]
traverse down..  ./net_data/mdl_spelling_0.bin
category..  [1]
traverse down..  ./net_data/mdl_spelling_0_1.bin
category..  [4]
traverse down..  ./net_data/mdl_spelling_0_1_4.bin
category..  [2]
traverse down..  ./net_data/mdl_spelling_0_1_4_2.bin
category..  [4]
traverse down..  ./net_data/mdl_spelling_0_1_4_2_4.bin
category..  [0]
key  debate  value  mdl_spelling_0_1_4_2_4_0


['debate']

In [ ]:
rt_name = "mdl_spelling"
dcMapping = {}
maxDepth = 30
btch_size = 128
wrl = WordManage()
wrl.set_words(sbst,0)
train_and_choose(rt_name,wrl,dcMapping,maxDepth,btch_size)
save_json(dcMapping,"word_tree_mapping.json")

In [ ]:
wrl.word_count()

In [ ]:
def remove_letter(wd):
    if len(wd) < 5:
        return wd
    cs = random.randint(0,len(wd)-1)
    return wd[0:cs] + wd[cs+1:]

def swap_letter(wd):
    if len(wd) < 3:
        return wd
    cs = random.randint(0,len(wd)-2)
    wd = [w for w in wd]
    t = wd[cs]
    wd[cs] = wd[cs+1]
    wd[cs+1] = t
    
    tmp = ""
    for w in wd:
        tmp += w
        
    return tmp

def change_letter(wd):    
    if len(wd) < 4:
        return wd
    
    alpha = ['a','b','c','d','e','f','g', \
        'h','i','j','k','l','m','n','o','p','q','r', \
        's','t','u','v','w','x','y','z']
    cs = random.randint(0,len(wd)-1)

    wd = [w for w in wd]
    
    wd[cs] = alpha[random.randint(0,25)]
    
    tmp = ""
    for w in wd:
        tmp += w
        
    return tmp

def drop_letter(wd):
    if len(wd) < 5:
        return wd
    cs = random.randint(0,len(wd)-2)
    
    wd = [w for w in wd]
    
    wd[cs] = ' '
    
    tmp = ""
    for w in wd:
        tmp += w
        
    return tmp   

def add_letter(wd):
    if len(wd) < 3:
        return wd
    
    #print("add letter")
    cs = random.randint(0,len(wd)-2)
    
    #wd = [w for w in wd]
    
    alpha = ['a','b','c','d','e','f','g', \
        'h','i','j','k','l','m','n','o','p','q','r', \
        's','t','u','v','w','x','y','z']
    
    #ln = len(wd)
    wd = wd[0:cs] + alpha[random.randint(0,25)] + wd[cs:] 
           
    return wd 


def word_mix(wd):
    fc_ar = [remove_letter,swap_letter,change_letter,drop_letter,add_letter]
    
    wd = fc_ar[random.randint(0,4)](wd)
    
    if random.random() < 0.4:
        wd = fc_ar[random.randint(0,4)](wd)
        
    if random.random() < 0.3:
        wd = fc_ar[random.randint(0,4)](wd)
        
    if random.random() < 0.2:
        wd = fc_ar[random.randint(0,4)](wd)
        
    return wd


In [ ]:
s = "transportation"
print(len(s))

for i in range(100):    
    nw_s = word_mix(s)
    print(nw_s,"  ",len(nw_s))


In [ ]:
for i in range(30):
    print(random.randint(0,4))

In [ ]:
vl = "nice"
        

In [ ]:
rtTreeName = "mdl_spelling"
avWords = wrl
wdDic = {}

In [ ]:
mdl = build_choose_and_train(avWords)

In [ ]:

# write the code to save the model..
# save_network(mdl,rtTreeName)

all_wrds = avWords.all_words()

#print(type(all_wrds),"      ",type(all_wrds[0]))

cat_w = get_category(mdl,all_wrds)

for i,wrd in enumerate(all_wrds):
    #print(wrd,"  ",cat_w[i])
    avWords.set_category(wrd,cat_w[i])
   

for i in range(5):
    sb_tree = rtTreeName + "_" + str(i)
    nw_wrl = avWords.filter_list(i)
    for w in nw_wrl.all_words():
        wdDic[w] = sb_tree
    
    print("category ",i," total word count ",len(nw_wrl.all_words()))
    # recurse..
    wrds = nw_wrl.all_words()
    if len(wrds) > 0:
        # recurse..
        print("recursing on ",sb_tree)
        # train_and_choose(sb_tree, nw_wrl, wdDic, maxDepth, btch_count

In [ ]:
wdDic

In [ ]:
save_network(mdl,"test")

In [ ]:
mdl = best_move(ImgNet(5))
load_data_into(mdl,"test")
#mdl = load_network("test")

In [ ]:
wrl.set_category('bike',2)
wrl.set_category('cause',2)
wrl.set_category('clerk',2)

In [ ]:
l = wrl.filter_list(2)

In [ ]:
# ['tension', 'session']
wd_e = encode_word("food")

In [ ]:
def display_im(im):
    plt.figure(figsize=(25, 150),dpi=80)
    plt.imshow(im)

In [ ]:
wd_m = encode_word("wood")

In [ ]:
kys = load_keys()